In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from random import randint
import numpy as np
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, Maximum, LayerNormalization
from keras.utils import *
from keras.initializers import *
import tensorflow as tf
import time, random
import sys

In [ ]:
from pathlib import Path

data_path = Path.cwd().parent / "content" / "drive" / "Shared drives" / "SigmaLaw" / "Data_W2V"
Data_path = Path.cwd().parent / "content" / "drive" / "Shared drives" / "SigmaLaw" / "classifier" / "data"

In [ ]:
!pip install stanfordcorenlp

In [ ]:
import json
from stanfordcorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP(r'/content/drive/Shared drives/SigmaLaw/stanford-corenlp-full-2018-10-05', quiet=False)
props = {'annotators': 'tokenize, ner, coref', 'pipelineLanguage': 'en'}

In [ ]:
DATA_PATH = Path.cwd().parent / "content" / "drive" / "Shared drives" / "SigmaLaw" / "GoogleNews-vectors-negative300.bin"

import gensim
model = gensim.models.KeyedVectors.load_word2vec_format(DATA_PATH / 'GoogleNews-vectors-negative300.bin', binary=True, unicode_errors='ignore')
from gensim.models import Word2Vec 

In [ ]:
def build_complete_ner(word, ner, l):
  if (len(l)>0):
    if (l[0].get('ner') == ner):
      word = word + " " + build_complete_ner(l[0].get("word"), l[0].get('ner'), l[1:len(l)])
    #print("built" + word)
    return word
  else:
    return ""

In [ ]:
def remove_non_ascii_1(text):
  return ''.join(i for i in text if ord(i)<128)

In [ ]:
def getMaskValues(ner_dict):
  masks_dict = {}
  keys = list(ner_dict.keys())
  for i in range(len(keys)):
    if ner_dict[keys[i]] == "P":
      value = 0.5*((1+i)/len(keys))
    elif ner_dict[keys[i]] == "O":
      value = 0.5 + 0.25*((1+i)/len(keys))
    elif ner_dict[keys[i]] == "L":
      value = 0.75 + 0.25*((1+i)/len(keys))
    masks_dict[keys[i]] = value
  return masks_dict

In [ ]:
def createVectorListFromToken(token_list, ner_list, masks_dict_list):

  max_length = 0
  vector_list = []
  for i in range(0,len(token_list)):
    sentence_vector = []
    current_token_list = token_list[i]
    current_ner_list = ner_list[i]
    mask_dict = masks_dict_list[i]

    assert len(current_token_list) == len(current_ner_list)

    for j in range(0, len(current_token_list)):
    
      if (current_ner_list[j] == "None"):
        try:
          vec = np.append(model[current_token_list[j]], 0)
         
        except KeyError:
          vec = np.zeros(301)
        
      else:
        try:
          vec = np.append(np.zeros(300), mask_dict[current_token_list[j]])
        except KeyError:    
          vec = np.zeros(301)        
        
      sentence_vector.append(vec)

    vector_list.append(sentence_vector)



  return vector_list

In [ ]:
def makeTokenNERListsFromParagraph(text):
  result = json.loads(nlp.annotate(text, properties=props))
  sentences = result['sentences']
  corefs = result['corefs']

  sentence_tokens_test = []
  sentence_ners_test = []
  ner_test = {}
  for each in sentences:
        
        q = each.get('tokens')

        for j in range(0,len(q)):
          sentence_tokens_test.append(q[j].get('word'))

          if (q[j].get("ner") == "PERSON" or q[j].get("ner") == "ORGANIZATION" or q[j].get("ner") == "LOCATION"):
            word = build_complete_ner(q[j].get("word"), q[j].get("ner"), q[j+1 : len(q)])
            if (q[j].get("ner") == "PERSON"):
              prefix = "P"
            elif (q[j].get("ner") == "ORGANIZATION") :
              prefix = "O"
            else:
              prefix = "L"
            if (j==0):
              ner_test[word] = prefix
              words = word.split(" ")
              for l in words:
                sentence_ners_test.append(prefix)
                
            if (j!=0 and q[j-1].get("ner") != q[j].get("ner")):
              ner_test[word] = prefix
              words = word.split(" ")
              for l in words:
                sentence_ners_test.append(prefix)
              
          else:
            sentence_ners_test.append("None")

  headwords_list = []
  for sentence in sentences:
    headwords = []
    for each in sentence['tokens']:
      if(each.get("ner") == "PERSON" or each.get("ner") == "ORGANIZATION" or each.get("ner") == "LOCATION"):
        headwords.append(each.get('word'))
      else:
        headwords.append(0)

    headwords_list.append(headwords)

  final_ner_dict = {}


  cluster_list = []
  for i in corefs.values():
    party_value = False
    for each in ner_test.keys():
      if each in i[0]['text']:
        party_value = True
        final_ner_dict[i[0]['text']] = ner_test[each]
    if party_value == True:
      cluster = []
      for j in i:
        # print(j)
        for index in range(j['startIndex']-1,j['endIndex']-1):
          # print(j['position'][0], index)
          headwords_list[j['position'][0]-1][index] = i[0]['text']
        cluster.append(j['text'])

      cluster_list.append(cluster)

  final_headwords = []
  for i in headwords_list:
    for j in i:
      final_headwords.append(j)

  for ner in ner_test.keys():
    val = True
    for cluster_ner in final_ner_dict.keys():
      if ner in cluster_ner:
        val = False
    if (val):
      final_ner_dict[ner] = ner_test[ner]






      
  return sentence_ners_test, final_headwords, final_ner_dict, sentence_tokens_test

In [ ]:
def Legal_entity_identifier(text):

  new_text = remove_non_ascii_1(text)

  ner_list, headwordsList, ner_dict, token_list = makeTokenNERListsFromParagraph(new_text)
  mask_dict = getMaskValues(ner_dict)

  vector_list = createVectorListFromToken([token_list],[ner_list],[mask_dict])

  d = {x: 0 for x in ner_dict}

  return new_text, d, headwordsList, vector_list

  


In [ ]:
f = open("/content/drive/Shared drives/SigmaLaw/data_2/54.txt", "rb")
text = f.read()
print(text)

b'Donaghy began his career as an NBA referee in September 1994 and continued in that position for thirteen seasons.  Battista agreed to pay Donaghy a fee for each game in which Donaghy correctly picked the winner. \xe2\x80\x82 Donaghy provided the picks to Martino, Martino relayed the information to Battista, and Battista placed the bets. \n'


In [ ]:
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [ ]:
def Probability_Cal(legal_entities,headwords_list,plaintif_prob,defendant_prob):
  plaintifs=legal_entities.copy()
  defendants=legal_entities.copy()
  count=legal_entities.copy()
  plaintifs_list=[]
  defendants_list=[]
  for i in range(0,len(headwords_list)):
    try: 
      plaintifs[headwords_list[i]]+=plaintif_prob[i]
      defendants[headwords_list[i]]+=defendant_prob[i]
      count[headwords_list[i]]+=1 
    except KeyError:  
        continue
    else:
        
       continue
  if (len(legal_entities)>0):
    le_list=list(legal_entities)
    for j in range(len(legal_entities)):
        try:
            p=plaintifs[le_list[j]]/count[le_list[j]]
            d=defendants[le_list[j]]/count[le_list[j]]
        except ZeroDivisionError:
            p=0
            d=0
        print(le_list[j]," :",p," , ",d)
        if(p>=0.5 and d<0.5):
            plaintifs_list.append(le_list[j])
        elif(d>=0.5 and p<0.5):
            defendants_list.append(le_list[j])
        elif(d<0.5 and p<0.5):
            continue
        elif(p>d):
            plaintifs_list.append(le_list[j])
        elif(d>p):
            defendants_list.append(le_list[j])



    return(plaintifs_list,defendants_list)

In [ ]:
model1= load_model(Data_path/"GRU_512")

In [ ]:
def findLegalParties():
    text=str(input("Enter the Legal Opinion Here: "))
    print("\n")
    new_text, legal_entities, headwords_list, vector_list = Legal_entity_identifier(text)
    max_length = 443
    new_list = []
    for i in vector_list: #padding the vectors
        for j in range(len(i),max_length):

            i.append(np.zeros(301))
        new_list.append(i)
    vector_array = np.array(new_list)
    k=max_length-len(headwords_list) #padding the tokens
    for n in range(k):
        headwords_list.append('0')
    dec=model1.predict(vector_array)
    dec_p=dec[:,:,0]
    dec_d=dec[:,:,1]
    probabilities_p = dec_p.reshape(max_length)
    probabilities_d = dec_d.reshape(max_length)
    bin_list_p = probabilities_p.tolist()
    bin_list_d = probabilities_d.tolist()
    print("Legal Entity : Petitioner Probability , Defendant Probability")
    p,d=Probability_Cal(legal_entities,headwords_list,bin_list_p,bin_list_d)
    print("\n")
    print("Petitioners:")
    for i in range(len(p)):
        print(p[i])
    print("\n") 
    print("Defendants:")
    for i in range(len(d)):
        print(d[i]) 
    return

In [ ]:
findLegalParties()

Enter the Legal Opinion Here: Ranjan Ramanayake was sentenced to 4 years of rigorous imprisonment on the basis of contempt of court regarding his comments on the corruption of the judiciary.


Legal Entity : Petitioner Probability , Defendant Probability
Ranjan Ramanayake  : 0.3350548499359623  ,  0.0005733267956694968


Petitioners:


Defendants:
